We'll use this script to compare lines from the two sites.

A reminder that for a double to be a greater value on PrizePicks than DraftKings, 
the two bets must multiply to a value lesser than three.

In [1]:
from draft_kings import get_draftkings_nba_data# this will run the script that scrapes the data into the json file we will use
import sys
sys.path.append('..')  # Add the parent directory to the sys.path
from constants import *
from pp_scrape import get_data

Getting DraftKings NBA data
<Response [200]>
487 Game Lines
1215 Player Points
583 Player Combos
1216 Player Rebounds
1217 Player Assists
1218 Player Threes
1157 Quick Hits
1293 Player Defense
1683 Combined Player Props
1684 Either Player Props
1320 Game Leaders
1280 Player Specials
1206 H2H Player Props
1686 Live Player Points
1690 Live Player Combos
1688 Live Player Rebounds
1687 Live Player Assists
1689 Live Player Threes
1691 Live Player Defense
697 Flash Props
520 Halves
522 Quarters
1682 Game Threes
521 Game Props
523 Team Props
519 Team Futures
1017 Wins/Playoffs
518 Awards
1175 Season Milestones
{487: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/42648/categories/487?format=json', 1215: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/42648/categories/1215?format=json', 583: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/42648/categories/583?format=json', 1216: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/even

SSLError: Failed to perform, curl: (35) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

In [20]:
import json, csv, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
COLLEGE_BASKETBALL = "20"
MLB = "2"
NBA = "7"

# get_draftkings_cbb_data() # this will run the script that scrapes the data into the json file we will use
# get_draftkings_nba_data()
get_data(NBA) # this will run the script that scrapes the data into the json file we will use

dict_keys(['adjusted_odds', 'board_time', 'custom_image', 'description', 'end_time', 'flash_sale_line_score', 'game_id', 'hr_20', 'in_game', 'is_live', 'is_promo', 'line_score', 'odds_type', 'projection_type', 'rank', 'refundable', 'start_time', 'stat_display_name', 'stat_type', 'status', 'today', 'tv_channel', 'updated_at'])
{'adjusted_odds': None, 'board_time': '2024-12-29T19:00:00-05:00', 'custom_image': None, 'description': 'GSW', 'end_time': None, 'flash_sale_line_score': None, 'game_id': 'NBA_game_ZfRcRSSwETSb6xQJvXqmPHXa', 'hr_20': True, 'in_game': False, 'is_live': False, 'is_promo': False, 'line_score': 28, 'odds_type': 'standard', 'projection_type': 'Fantasy Score', 'rank': 0, 'refundable': True, 'start_time': '2025-01-09T19:10:00-05:00', 'stat_display_name': 'Fantasy Score', 'stat_type': 'Fantasy Score', 'status': 'pre_game', 'today': True, 'tv_channel': None, 'updated_at': '2025-01-09T15:17:56-05:00'}
dict_keys(['adjusted_odds', 'board_time', 'custom_image', 'description', 

[{'name': 'Jalen Duren',
  'opponent': 'GSW',
  'Fantasy Score': 28,
  'Rebounds': 9.5,
  'Pts+Rebs+Asts': 21.5,
  'Dunks': 2.5,
  'Points': 9.5,
  'Pts+Rebs': 19.5,
  'Pts+Asts': 11.5,
  'Rebs+Asts': 11.5,
  'Defensive Rebounds': 7,
  'Offensive Rebounds': 3.5},
 {'name': 'Scottie Barnes',
  'opponent': 'CLE',
  'Fantasy Score': 37.5,
  'Points': 17.5,
  'FG Attempted': 15.5,
  'Rebs+Asts': 12.5,
  'Pts+Rebs+Asts': 30.5,
  'Assists': 5,
  'Dunks': 0.5,
  'Rebounds': 7.5,
  'FG Made': 7.5,
  'Pts+Rebs': 25.5,
  'Pts+Asts': 23.5},
 {'name': 'Immanuel Quickley',
  'opponent': 'CLE',
  'Fantasy Score': 29,
  'Points': 15.5,
  'Rebs+Asts': 9,
  'FG Attempted': 12.5,
  'Pts+Asts': 21,
  'FG Made': 5,
  'Pts+Rebs+Asts': 24.5,
  'Pts+Rebs': 18.5,
  'Rebounds': 3.5,
  '3-PT Attempted': 6,
  'Free Throws Made': 3},
 {'name': 'RJ Barrett',
  'opponent': 'CLE',
  'Fantasy Score': 34,
  'Pts+Rebs': 25.5,
  'FG Attempted': 15.5,
  'Pts+Asts': 24.5,
  '3-PT Made': 1.5,
  'FG Made': 7.5,
  'Points': 

In [22]:
dk_lines = pd.read_csv('draftkings_nba.csv', encoding='latin-1')
# print(dk_lines['category'].value_counts())
# # list the items that appear multiple times
print(dk_lines['category'].value_counts()[dk_lines['category'].value_counts() == 1])
game_props = ["Game Lines", "Team Props", "Halves", "Game Props", "Total", "Innings", "Total Runs - 1st 5 Innings", "Quick Hits", "Game Leaders", "Game Lines", "Total", "Most Points", "Moneyline", "Halves", "Total 1st Half", "Moneyline 1st Half", "Quarters", "Total 1st Quarter", "Moneyline 1st Quarter", "Q1 Player Props"]


player_lines = dk_lines[~dk_lines['category'].isin(game_props)]
player_lines = player_lines[~player_lines['participants'].isin(game_props)]
print(player_lines['category'].value_counts())

# drop na just for the 'category' label
player_lines = player_lines.dropna( subset=['category'] )
#drop the entire column for 'prop_label' since it is not needed
player_lines = player_lines.drop(columns=['prop_label'])
player_lines

category
Donte DiVincenzo Points                          1
Deandre Ayton Assists                            1
Stephen Curry Assists                            1
Tim Hardaway Jr. Assists                         1
Tobias Harris Assists                            1
                                                ..
Bradley Beal Points + Rebounds + Assists         1
Bogdan Bogdanovic Points + Rebounds + Assists    1
Jaren Jackson Jr. Points + Rebounds + Assists    1
Jalen Green Points + Rebounds + Assists          1
Toumani Camara Rebounds                          1
Name: count, Length: 477, dtype: int64
category
Anthony Black Points                         1
Buddy Hield Assists                          1
Deandre Ayton Assists                        1
Anfernee Simons Assists                      1
Trayce Jackson-Davis Assists                 1
                                            ..
Jae'Sean Tate Points + Rebounds + Assists    1
Ja Morant Points + Rebounds + Assists        1
Fred V

,providerOutcomeId,providerId,providerOfferId,label,oddsAmerican,oddsDecimal,oddsDecimalDisplay,oddsFractional,line,participant,participantType,main,sortOrder,tags,participants,category
48,0QA228822097#128907016_13L42648Q1243631508Q20,2,228822097,10+,-120,1.833333,1.83,5/6,Anthony Black,Unknown,True,18316,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '713759', 'name': 'Anthony Black', 'ty...",Player Points,Anthony Black Points
55,0QA228822102#128907001_13L42648Q11520391110Q20,2,228822102,25+,-115,1.869565,1.86,20/23,Anthony Edwards,Unknown,True,18601,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '509114', 'name': 'Anthony Edwards', '...",Player Points,Anthony Edwards Points
63,0QA228822095#128906949_13L42648Q1-1088221744Q20,2,228822095,18+,110,2.100000,2.10,11/10,Cole Anthony,Unknown,True,21049,"['MostBalancedOdds', 'SGP']","[{'id': '527225', 'name': 'Cole Anthony', 'typ...",Player Points,Cole Anthony Points
71,0QA228822099#128907017_13L42648Q12000308824Q20,2,228822099,12+,105,2.050000,2.05,21/20,Donte DiVincenzo,Unknown,True,20517,"['MostBalancedOdds', 'SGP']","[{'id': '33185', 'name': 'Donte DiVincenzo', '...",Player Points,Donte DiVincenzo Points
77,0QA228822109#128907060_13L42648Q12000308824Q20,2,228822109,12+,-120,1.833333,1.83,5/6,Goga Bitadze,Unknown,True,18360,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '33097', 'name': 'Goga Bitadze', 'type...",Player Points,Goga Bitadze Points
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2662,0QA228824582#128918000_13L42648Q1160400555Q20,2,228824582,2+,-105,1.952381,1.95,20/21,Jaren Jackson Jr.,Unknown,True,19500,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '33142', 'name': 'Jaren Jackson Jr.', ...",Player Defense,Jaren Jackson Jr. Blocks
2664,0QA228828627#128953949_13L42648Q11130272651Q20,2,228828627,1+,-180,1.555556,1.55,5/9,Clint Capela,Unknown,True,15549,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '33076', 'name': 'Clint Capela', 'type...",Player Defense,Clint Capela Blocks
2667,0QA228828587#128953793_13L42648Q1160400555Q20,2,228828587,2+,175,2.750000,2.75,7/4,Kevin Durant,Unknown,True,27593,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '32896', 'name': 'Kevin Durant', 'type...",Player Defense,Kevin Durant Blocks
2668,0QA228830005#128970319_13L42648Q11130272651Q20,2,228830005,1+,-205,1.487805,1.48,20/41,Bam Adebayo,Unknown,True,14819,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '33161', 'name': 'Bam Adebayo', 'type'...",Player Defense,Bam Adebayo Blocks


In [23]:
def get_categories(dk_db):
    ans = []
    combos = []
    for index, row in dk_db.iterrows():
        # print(row['category'], row['participant'], row['line'])
        if row['line'] in row['category']:
            stat_tracked = row['category'].replace(row['line'], '').strip()
            if stat_tracked not in ans:
                if "+" in stat_tracked or " - " in stat_tracked:
                    combos.append(stat_tracked)
                ans.append(stat_tracked)
    return ans, combos



In [24]:
categories, combos = get_categories(player_lines)
player_lines['stat_tracked'] = player_lines.apply(lambda player: next((category.lower() for category in combos if category in player['category']), None), axis=1)
player_lines['stat_tracked'] = player_lines.apply(lambda player: next((category.lower() for category in categories if category in player['category'] and "+" not in player['category']), player['stat_tracked']), axis=1)

player_lines = player_lines.dropna()
print(categories)
print(combos)


print(player_lines['stat_tracked'].value_counts())
print(player_lines.shape)  

# this will show us the most common stats that are tracked
player_lines = player_lines[~player_lines['stat_tracked'].str.contains(':')]
player_lines = player_lines[~player_lines['stat_tracked'].str.contains('1st')]
player_lines = player_lines[~player_lines['category'].str.contains('Live')]
# player_lines = player_lines[player_lines['label'].isin(['Over', 'Under'])]

# print(player_lines['stat_tracked'].value_counts())
# print(player_lines['label'].value_counts())
player_lines.head()

['Points', 'Points + Rebounds + Assists', 'Rebounds', 'Assists', 'Three Pointers Made', 'Blocks']
['Points + Rebounds + Assists']
stat_tracked
points                         97
rebounds                       97
assists                        93
points + rebounds + assists    92
three pointers made            82
blocks                         16
Name: count, dtype: int64
(477, 17)


,providerOutcomeId,providerId,providerOfferId,label,oddsAmerican,oddsDecimal,oddsDecimalDisplay,oddsFractional,line,participant,participantType,main,sortOrder,tags,participants,category,stat_tracked
48,0QA228822097#128907016_13L42648Q1243631508Q20,2,228822097,10+,-120,1.833333,1.83,5/6,Anthony Black,Unknown,True,18316,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '713759', 'name': 'Anthony Black', 'ty...",Player Points,Anthony Black Points,points
55,0QA228822102#128907001_13L42648Q11520391110Q20,2,228822102,25+,-115,1.869565,1.86,20/23,Anthony Edwards,Unknown,True,18601,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '509114', 'name': 'Anthony Edwards', '...",Player Points,Anthony Edwards Points,points
63,0QA228822095#128906949_13L42648Q1-1088221744Q20,2,228822095,18+,110,2.100000,2.10,11/10,Cole Anthony,Unknown,True,21049,"['MostBalancedOdds', 'SGP']","[{'id': '527225', 'name': 'Cole Anthony', 'typ...",Player Points,Cole Anthony Points,points
71,0QA228822099#128907017_13L42648Q12000308824Q20,2,228822099,12+,105,2.050000,2.05,21/20,Donte DiVincenzo,Unknown,True,20517,"['MostBalancedOdds', 'SGP']","[{'id': '33185', 'name': 'Donte DiVincenzo', '...",Player Points,Donte DiVincenzo Points,points
77,0QA228822109#128907060_13L42648Q12000308824Q20,2,228822109,12+,-120,1.833333,1.83,5/6,Goga Bitadze,Unknown,True,18360,"['MostBalancedOdds', 'SGP', 'MostBalancedGloba...","[{'id': '33097', 'name': 'Goga Bitadze', 'type...",Player Points,Goga Bitadze Points,points


In [25]:
player_lines['stat_tracked'].value_counts()
#remove all lines with 

stat_tracked
points                         96
rebounds                       96
assists                        92
points + rebounds + assists    91
three pointers made            82
blocks                         15
Name: count, dtype: int64

In [26]:
final_dk_db = {} #dic so we can combine lines for each player

for index, row in player_lines.iterrows():
    print(row['line'], row['stat_tracked'], row['label'])
    if row['line'] not in final_dk_db:
        final_dk_db[row['line']] = {
            'name': row['line']
        }
    over_or_under = "Over" if "+" in row['label'] else "Under"
    final_dk_db[row['line']][row['stat_tracked']] = float(row['label'].replace("+", '').replace("-", '')) - 0.5 if over_or_under == "Over" else float(row['label'].replace("+", '').replace("-", '')) + 0.5
    final_dk_db[row['line']][row['stat_tracked'] + '_' + over_or_under] = row['oddsDecimal']


final_dk_db = pd.DataFrame(final_dk_db).T



Anthony Black points 10+
Anthony Edwards points 25+
Cole Anthony points 18+
Donte DiVincenzo points 12+
Goga Bitadze points 12+
Jaden McDaniels points 10+
Jett Howard points 7+
Jonathan Isaac points 8+
Julius Randle points 18+
Kentavious Caldwell-Pope points 12+
Mike Conley points 8+
Naz Reid points 12+
Nickeil Alexander-Walker points 6+
Rudy Gobert points 10+
Tristan da Silva points 12+
Wendell Carter Jr. points 10+
Caris LeVert points 13+
Darius Garland points 21+
Dean Wade points 8+
Evan Mobley points 20+
Gradey Dick points 12+
Immanuel Quickley points 16+
Ja'Kobe Walter points 8+
Jakob Poeltl points 12+
Jamal Shead points 6+
Jarrett Allen points 16+
Kelly Olynyk points 8+
Max Strus points 12+
RJ Barrett points 20+
Scottie Barnes points 20+
Buddy Hield points 15+
Cade Cunningham points 25+
Dennis Schroder points 15+
Draymond Green points 10+
Isaiah Stewart points 7+
Jalen Duren points 10+
Malik Beasley points 17+
Stephen Curry points 27+
Tim Hardaway Jr. points 12+
Tobias Harris poi

In [27]:
final_dk_db

,name,points,points_Over,points + rebounds + assists,points + rebounds + assists_Over,rebounds,rebounds_Over,assists,assists_Over,three pointers made,three pointers made_Over,blocks,blocks_Over
Anthony Black,Anthony Black,9.5,1.833333,15.5,1.769231,2.5,1.689655,3.5,2.05,0.5,1.740741,NaN,NaN
Anthony Edwards,Anthony Edwards,24.5,1.869565,34.5,1.909091,5.5,1.869565,4.5,2.3,3.5,1.952381,NaN,NaN
Cole Anthony,Cole Anthony,17.5,2.1,26.5,1.909091,4.5,1.909091,5.5,2.35,1.5,1.740741,NaN,NaN
Donte DiVincenzo,Donte DiVincenzo,11.5,2.05,19.5,2.2,3.5,1.769231,3.5,2.05,2.5,2.0,NaN,NaN
Goga Bitadze,Goga Bitadze,11.5,1.833333,24.5,1.909091,10.5,2.2,2.5,1.769231,0.5,3.95,1.5,1.869565
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lauri Markkanen,Lauri Markkanen,24.5,1.952381,32.5,1.869565,6.5,1.952381,1.5,1.625,3.5,1.909091,NaN,NaN
Nikola Jovic,Nikola Jovic,11.5,2.05,18.5,1.909091,4.5,2.0,2.5,1.645161,1.5,2.05,NaN,NaN
Terry Rozier,Terry Rozier,14.5,1.8,22.5,1.869565,4.5,2.1,3.5,2.0,1.5,1.588235,NaN,NaN
Tyler Herro,Tyler Herro,24.5,1.833333,35.5,1.833333,5.5,2.0,5.5,1.869565,4.5,2.5,NaN,NaN


In [28]:
prizepicks = pd.read_csv("pp_lines.csv")

merged = pd.merge(prizepicks, final_dk_db, on='name')
#get a list of all the column names
cols = merged.columns.tolist()
pp_to_dk_stats.keys()


dict_keys(['Rebounds', 'Points', 'Pts+Asts', 'Pts+Rebs', 'Pts+Rebs+Asts', 'Assists', 'Rebs+Asts', 'Fantasy Score', '3-PT Made', 'Hits Allowed', 'Pitcher Strikeouts', 'Pitching Outs', 'Walks Allowed', 'Hits+Runs+RBIS', 'Hitter Strikeouts', 'Total Bases', 'Runs', 'Walks'])

In [29]:
results = []

for index, row in merged.iterrows():
    # Access the values of each column in the current row
    name = row['name']

    for pp_stat_type in pp_to_dk_stats.keys():
        # failsafe 
        if pp_stat_type not in row:
            continue

        if float(row[pp_stat_type]) <= 0:
            continue

        if pp_to_dk_stats[pp_stat_type] not in row:
            continue
    
        if float(row[pp_to_dk_stats[pp_stat_type]]) <= 0:
            continue

        # Create a dictionary to store the result
        result_dict = {
            'name': name,
            'stat': pp_stat_type,
            'line': row[pp_stat_type],
            'dk_line': float(row[pp_to_dk_stats[pp_stat_type]]),
            'dk_over_line': row[pp_to_dk_stats[pp_stat_type] + '_Over'],
            'dk_under_line': row[pp_to_dk_stats[pp_stat_type] + '_Under'] if pp_to_dk_stats[pp_stat_type] + '_Under' in row else 100,
        }

        # Get the dk stat type that corresponds to the current PP stat type
        dk_stat_type = pp_to_dk_stats[pp_stat_type]

        # Get the value of the FBref stat type for the current player
        dk_stat_value = row[dk_stat_type]



        # Append the dictionary to the results list
        results.append(result_dict)

results_df = pd.DataFrame(results)
print(results_df.head())


results_df = results_df.dropna()
# results_df = results_df.dropna() 
# # get data types for each column 
# print(results_df.dtypes)



          name           stat  line  dk_line  dk_over_line  dk_under_line
0  Jalen Duren       Rebounds   9.5     10.5      2.150000            100
1  Jalen Duren         Points   9.5      9.5      1.909091            100
2  Jalen Duren  Pts+Rebs+Asts  21.5     21.5      1.909091            100
3  Jalen Duren        Assists   NaN      2.5      2.600000            100
4  Jalen Duren      3-PT Made   NaN      NaN           NaN            100


In [30]:
from scipy.stats import poisson
from scipy.optimize import minimize

def implied_odds_at_point(draft_kings_line, lambda_):
        # DraftKings line is an integer threshold (either x.5 or x.0, where x.0 has a push opportunity, so be wary)
        # use the pdf function to get the exact probability of the line
        exact_odds = poisson.pmf(draft_kings_line, lambda_)
        return ((1 - poisson.cdf(draft_kings_line, lambda_)) / (1-exact_odds))


def get_implied_odds(pp_line, draft_kings_line, draft_kings_over_line):
    # Function to solve for lambda
    def solve_lambda(lambda_):
        return abs(implied_odds_at_point(draft_kings_line, lambda_) - (1/draft_kings_over_line))

    # Optimize to find lambda
    # essentially, were setting the two sides of the minus sign to be equal
    result = minimize(solve_lambda, x0=[draft_kings_line], bounds=[(0, None)]) 
    lambda_ = result.x[0]
    print(lambda_, pp_line, draft_kings_line, draft_kings_over_line)
    
    # Calculate the probability of exceeding the pp_line
    prob_over_pp_line = (1 - poisson.cdf(pp_line, lambda_)) / (1 - poisson.pmf(pp_line, lambda_))
    return prob_over_pp_line

results_df["implied_odds"] = results_df.apply(lambda row: get_implied_odds(row["line"], row["dk_line"], row["dk_over_line"]), axis=1)

10.38584669710156 9.5 10.5 2.15
9.855051479840872 9.5 9.5 1.90909091
21.94638404948603 21.5 21.5 1.90909091
8.061249380634738 7.5 7.5 1.8
19.05544113328036 17.5 19.5 2.25
31.446811758395633 30.5 30.5 1.8
5.403921143273593 5.0 5.5 2.2
3.9442124028444443 3.5 3.5 1.8
16.016423019310537 15.5 15.5 1.86956522
25.36708899605177 24.5 24.5 1.8
6.447018821441663 6.0 6.5 2.15
20.05795640745954 20.0 19.5 1.86956522
30.998901543776768 30.5 30.5 1.90909091
1.7175824771235049 1.5 1.5 1.95238096
12.319976967649543 12.5 11.5 1.74074075
17.243797985750305 17.0 16.5 1.8
10.569234069377343 10.0 10.5 2.05
16.016423019310537 15.5 15.5 1.86956522
29.050310489736848 27.5 29.5 2.2
21.42214954040618 21.0 20.5 1.76923077
31.15427474841054 30.5 30.5 1.86956522
6.967629225010736 7.0 6.5 1.83333334
9.319010303532009 9.0 9.5 2.2
20.403889469787245 20.0 19.5 1.76923077
34.1547031687696 32.5 34.5 2.15
11.372554629968365 9.5 11.5 2.15
10.300863776175412 9.0 10.5 2.2
11.873049273332185 11.0 11.5 1.90909091
23.9588679008

In [31]:
results_df.sort_values(by="implied_odds", ascending=False)

,name,stat,line,dk_line,dk_over_line,dk_under_line,implied_odds
41,Max Strus,Points,9.5,11.5,2.150000,100,0.698517
67,Dean Wade,Pts+Rebs+Asts,12.5,14.5,2.100000,100,0.683518
61,Kelly Olynyk,Points,6.5,7.5,1.909091,100,0.666087
241,Klay Thompson,Points,15.5,17.5,2.100000,100,0.665486
336,Clint Capela,Points,9.5,11.5,2.350000,100,0.663836
...,...,...,...,...,...,...,...
192,Ja Morant,Pts+Rebs+Asts,32.5,31.5,1.909091,100,0.453469
87,Dennis Schroder,Pts+Rebs+Asts,23.5,22.5,1.869565,100,0.452013
156,Naz Reid,Points,12.5,11.5,1.800000,100,0.441295
427,Nickeil Alexander-Walker,Pts+Rebs+Asts,10.5,9.5,1.769231,100,0.440323


In [32]:
results_df = results_df[results_df['line'] == results_df['dk_line']]
results_df.sort_values(by='dk_over_line', ascending=True)

,name,stat,line,dk_line,dk_over_line,dk_under_line,implied_odds
284,Devin Booker,3-PT Made,2.5,2.5,1.689655,100,0.591837
168,Wendell Carter Jr.,Assists,1.5,1.5,1.740741,100,0.574468
174,Anthony Black,3-PT Made,0.5,0.5,1.740741,100,0.574468
324,Tyus Jones,3-PT Made,1.5,1.5,1.740741,100,0.574468
304,Bradley Beal,3-PT Made,1.5,1.5,1.740741,100,0.574468
...,...,...,...,...,...,...,...
183,Desmond Bane,Assists,5.5,5.5,2.050000,100,0.487805
182,Desmond Bane,Pts+Rebs+Asts,29.5,29.5,2.050000,100,0.487805
157,Naz Reid,Pts+Rebs+Asts,19.5,19.5,2.050000,100,0.487805
88,Dennis Schroder,Assists,5.5,5.5,2.100000,100,0.476190


In [33]:
results_df.sort_values(by='dk_over_line', ascending=False)

,name,stat,line,dk_line,dk_over_line,dk_under_line,implied_odds
253,Scoot Henderson,Assists,4.5,4.5,2.150000,100,0.465116
88,Dennis Schroder,Assists,5.5,5.5,2.100000,100,0.476190
157,Naz Reid,Pts+Rebs+Asts,19.5,19.5,2.050000,100,0.487805
183,Desmond Bane,Assists,5.5,5.5,2.050000,100,0.487805
182,Desmond Bane,Pts+Rebs+Asts,29.5,29.5,2.050000,100,0.487805
...,...,...,...,...,...,...,...
304,Bradley Beal,3-PT Made,1.5,1.5,1.740741,100,0.574468
273,Toumani Camara,Assists,1.5,1.5,1.740741,100,0.574468
174,Anthony Black,3-PT Made,0.5,0.5,1.740741,100,0.574468
168,Wendell Carter Jr.,Assists,1.5,1.5,1.740741,100,0.574468


In [34]:
import math 
math.sqrt(3)


1.7320508075688772

In [35]:
1 / math.sqrt(3)

0.5773502691896258

In [36]:
1 - (0.487 - (115 / 210 - 0.5))

0.5606190476190477